In [ ]:
%pip install torch

import json
import os
from datetime import datetime
from collections import defaultdict
import statistics

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load your NER model
model_path = "final_amharic_ner_model"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()


  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB 81.9 kB/s eta 0:43:57
   ---------------------------------------- 0.0/216.1 MB 81.9 kB/s eta 0:43:57
   ---------------------------------------- 0.0/216.1 MB 81.9 kB/s eta 0:43:57
   ---------------------------------------- 0.1/216.1 MB 142.6 kB/s eta 0:25:15

ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\rebika\Desktop\amharic-telegram-ner\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\rebika\Desktop\amharic-telegram-ner\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\rebika\Desktop\amharic-telegram-ner\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\rebika\Desktop\amharic-telegram-ner\venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 102, in read
    self.__buf.write(data)
  File "C:\Users\rebika\AppData\Local\Programs\Python\Python312\Lib\tempfile.py", line 499, in func_wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 

ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Check out the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
def extract_entities(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, return_offsets_mapping=True)
    with torch.no_grad():
        output = model(**tokens).logits
    predictions = torch.argmax(output, dim=-1).squeeze().tolist()
    words = tokenizer.convert_ids_to_tokens(tokens["input_ids"].squeeze())
    labels = [model.config.id2label[p] for p in predictions]

    prices = []
    products = []

    current_price = []
    current_product = []

    for token, label in zip(words, labels):
        if label.startswith("B-PRICE"):
            if current_price:
                prices.append("".join(current_price))
            current_price = [token]
        elif label.startswith("I-PRICE"):
            current_price.append(token)
        else:
            if current_price:
                prices.append("".join(current_price))
                current_price = []

        if label.startswith("B-Product"):
            if current_product:
                products.append("".join(current_product))
            current_product = [token]
        elif label.startswith("I-Product"):
            current_product.append(token)
        else:
            if current_product:
                products.append("".join(current_product))
                current_product = []

    if current_price:
        prices.append("".join(current_price))
    if current_product:
        products.append("".join(current_product))

    return products, prices


In [ ]:
def analyze_vendor(channel_name, file_path):
    with open(file_path, encoding='utf-8') as f:
        posts = json.load(f)

    total_views = 0
    prices = []
    post_dates = []
    highest_view = 0
    top_post = None

    for post in posts:
        msg = post.get("message", "")
        views = post.get("views", 0)
        timestamp = post.get("date")

        if not msg.strip(): continue
        total_views += views

        # Extract entities
        products, post_prices = extract_entities(msg)
        prices.extend([float(p.replace(",", "").replace("ብር", "")) for p in post_prices if p.replace(",", "").isdigit()])

        if views > highest_view:
            highest_view = views
            top_post = {
                "text": msg,
                "views": views,
                "product": products,
                "price": post_prices
            }

        if timestamp:
            post_dates.append(datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S"))

    # Posts per week
    if not post_dates:
        return None  # No valid posts

    total_posts = len(post_dates)
    weeks = (max(post_dates) - min(post_dates)).days / 7.0
    posts_per_week = total_posts / weeks if weeks > 0 else total_posts

    avg_views = total_views / total_posts if total_posts else 0
    avg_price = sum(prices) / len(prices) if prices else 0

    lending_score = (0.5 * avg_views) + (0.3 * posts_per_week) + (0.2 * (avg_price / 1000))

    return {
        "Channel": channel_name,
        "Posts/Week": round(posts_per_week, 2),
        "Avg. Views/Post": round(avg_views, 2),
        "Avg. Price (ETB)": round(avg_price, 2),
        "Lending Score": round(lending_score, 2),
        "Top Post": top_post
    }


In [ ]:
results = []
data_dir = "data/raw"

for filename in os.listdir(data_dir):
    if filename.endswith(".json"):
        channel = filename.replace(".json", "")
        file_path = os.path.join(data_dir, filename)
        res = analyze_vendor(channel, file_path)
        if res:
            results.append(res)


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df_sorted = df.sort_values("Lending Score", ascending=False)

print(df_sorted[["Channel", "Avg. Views/Post", "Posts/Week", "Avg. Price (ETB)", "Lending Score"]])


In [ ]:
df_sorted.to_csv("vendor_scorecard.csv", index=False)
